In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

In [2]:
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm
from qdrant_client import QdrantClient, models

In [3]:
dataset = load_from_disk("../data/processed/arcd_clean")
train = dataset["train"]
answers = [a["text"][0] if isinstance(a["text"], list) else a["text"]
            for a in train["answers"]]

print(f"Total answers: {len(answers)}")
print("Example:", answers[0])

Total answers: 693
Example: صحفي واعلامي


In [4]:
models_to_test = {
    "triplet": "Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2",
    "e5_base": "intfloat/multilingual-e5-base"
}

query = "ما هي عاصمة فلسطين؟"

In [5]:
def test_model(model_name, answers, query, limit=3):
    print(f"\n🔻 Testing model: {model_name}")
    model = SentenceTransformer(model_name)
    
    # generate embeddings
    embeddings = model.encode(
        answers,
        show_progress_bar=True,
        batch_size=16,
        convert_to_numpy=True,
        normalize_embeddings=True,
    )
    print("Embeddings shape:", embeddings.shape)

    # build Qdrant collection
    client = QdrantClient(host="localhost", port=6333)
    collection_name = "temp_test_collection"
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=embeddings.shape[1],
            distance=models.Distance.COSINE
        ),
    )

    payloads = [{"answer_text": ans, "id": i} for i, ans in enumerate(answers)]
    client.upsert(
        collection_name=collection_name,
        points=models.Batch(
            ids=list(range(len(embeddings))),
            vectors=embeddings.tolist(),
            payloads=payloads,
        ),
    )

    # query
    query_vec = model.encode(["query: " + query], normalize_embeddings=True)
    results = client.search(
        collection_name=collection_name,
        query_vector=query_vec[0],
        limit=limit
    )

    print(f"\nQuery: {query}")
    for r in results:
        print("→", r.payload["answer_text"], "| Score:", round(r.score, 3))

    # return embeddings
    return model, embeddings

In [6]:
results_dict = {}

for key, model_name in models_to_test.items():
    model, embeddings = test_model(model_name, answers, query)
    results_dict[key] = {
        "model_name": model_name,
        "model_obj": model,
        "embeddings": embeddings
    }


🔻 Testing model: Omartificial-Intelligence-Space/Arabic-Triplet-Matryoshka-V2


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Embeddings shape: (693, 768)


/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/713825613.py:18: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/713825613.py:38: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(



Query: ما هي عاصمة فلسطين؟
→ معظم اراضي فلسطين ولبنان بما فيها مدينة القدس | Score: 0.498
→ فلسطين ولبنان | Score: 0.413
→ هي القاهرة الكبري | Score: 0.319

🔻 Testing model: intfloat/multilingual-e5-base


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Embeddings shape: (693, 768)

Query: ما هي عاصمة فلسطين؟
→ معظم اراضي فلسطين ولبنان بما فيها مدينة القدس | Score: 0.825
→ فلسطين ولبنان | Score: 0.815
→ وفلسطين المحتلة اسراييل | Score: 0.794


### Final Embedding Generation and Storage (Best Model)

In [7]:
best_model_name = "intfloat/multilingual-e5-base"
model = SentenceTransformer(best_model_name)

client = QdrantClient(host="localhost", port=6333)

# generate and store ANSWER embeddings
answers = [a["text"][0] if isinstance(a["text"], list) else a["text"]
            for a in train["answers"]]

ans_embeddings = model.encode(
    answers,
    show_progress_bar=True,
    batch_size=16,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

client.recreate_collection(
    collection_name="arcd_answers",
    vectors_config=models.VectorParams(
        size=ans_embeddings.shape[1],
        distance=models.Distance.COSINE
    ),
)

payloads = [{"answer_text": ans, "id": i} for i, ans in enumerate(answers)]
client.upsert(
    collection_name="arcd_answers",
    points=models.Batch(
        ids=list(range(len(ans_embeddings))),
        vectors=ans_embeddings.tolist(),
        payloads=payloads,
    ),
)

print(f"Stored {len(ans_embeddings)} answer embeddings in Qdrant.")

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/3469084275.py:18: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Stored 693 answer embeddings in Qdrant.


In [8]:
# generate and store CONTEXT embeddings
contexts = [c for c in train["context"]]

ctx_embeddings = model.encode(
    contexts,
    show_progress_bar=True,
    batch_size=16,
    convert_to_numpy=True,
    normalize_embeddings=True,
)

client.recreate_collection(
    collection_name="arcd_contexts",
    vectors_config=models.VectorParams(
        size=ctx_embeddings.shape[1],
        distance=models.Distance.COSINE
    ),
)

payloads = [{"context_text": ctx, "id": i} for i, ctx in enumerate(contexts)]
client.upsert(
    collection_name="arcd_contexts",
    points=models.Batch(
        ids=list(range(len(ctx_embeddings))),
        vectors=ctx_embeddings.tolist(),
        payloads=payloads,
    ),
)

print(f"Stored {len(ctx_embeddings)} context embeddings in Qdrant.")

Batches:   0%|          | 0/44 [00:00<?, ?it/s]

/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/52649018.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Stored 693 context embeddings in Qdrant.


In [9]:
query = "ما هي عاصمة فلسطين؟"
query_vec = model.encode(["query: " + query], normalize_embeddings=True)

print("\n🔻From Answers:")
for r in client.search("arcd_answers", query_vector=query_vec[0], limit=3):
    print("→", r.payload["answer_text"], "| Score:", round(r.score, 3))

print("\n🔻From Contexts:")
for r in client.search("arcd_contexts", query_vector=query_vec[0], limit=3):
    print("→", r.payload["context_text"][:200], "| Score:", round(r.score, 3))


🔻From Answers:
→ معظم اراضي فلسطين ولبنان بما فيها مدينة القدس | Score: 0.825
→ فلسطين ولبنان | Score: 0.815
→ وفلسطين المحتلة اسراييل | Score: 0.794

🔻From Contexts:


/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/2517720341.py:5: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  for r in client.search("arcd_answers", query_vector=query_vec[0], limit=3):
/var/folders/jv/c9vh4bnj087cpqc52674mcsh0000gn/T/ipykernel_2942/2517720341.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  for r in client.search("arcd_contexts", query_vector=query_vec[0], limit=3):


→ فلسطين بالعبرية او حسب السياق باليونانية هي منطقة جغرافية في غرب اسيا عادة ما تشمل اسراييل والضفة الغربية وقطاع غزة وفي بعض التعاريف بعض اجزاء من غرب الاردن مشكلة الجزء الجنوبي الغربي من بلاد الشام تق | Score: 0.81
→ فلسطين بالعبرية او حسب السياق باليونانية هي منطقة جغرافية في غرب اسيا عادة ما تشمل اسراييل والضفة الغربية وقطاع غزة وفي بعض التعاريف بعض اجزاء من غرب الاردن مشكلة الجزء الجنوبي الغربي من بلاد الشام تق | Score: 0.81
→ فلسطين بالعبرية او حسب السياق باليونانية هي منطقة جغرافية في غرب اسيا عادة ما تشمل اسراييل والضفة الغربية وقطاع غزة وفي بعض التعاريف بعض اجزاء من غرب الاردن مشكلة الجزء الجنوبي الغربي من بلاد الشام تق | Score: 0.81
